####### we need to create a sql database in the Azure account 

In [0]:
%sql
-- 1- create bronze schema in Hive Metastore 
create schema if not exists hive_metastore.default;

In [0]:
%sql
-- 2- create External Table 

In [0]:
%sql
-- without Azure 
-- Tell Databricks to use Unity Catalog, not Hive
USE CATALOG gizmobox;
USE SCHEMA bronze;   -- or: USE SCHEMA gizmobox.bronze;

-- Create the managed Delta table
CREATE OR REPLACE TABLE gizmobox.bronze.refunds (
  refund_id INT,
  payment_id INT,
  refund_timestamp TIMESTAMP,
  refund_amount DECIMAL(10,2),
  refund_reason STRING
) USING DELTA;

-- Load your sample rows
INSERT INTO gizmobox.bronze.refunds
(refund_id, payment_id, refund_timestamp, refund_amount, refund_reason) VALUES
(1, 66,'2025-01-10 11:30:00',  85.75,'Payment Error:Retailer'),
(2, 69,'2025-01-01 12:40:15', 120.50,'Order Cancelled:Customer'),
(3, 72,'2025-01-06 14:45:30',  65.00,'Product Returned:Customer'),
(4, 73,'2025-01-07 16:10:45', 210.99,'Order Cancelled:Customer'),
(5, 75,'2025-01-09 18:25:00',  45.20,'Payment Error:Retailer'),
(6, 80,'2025-01-10 09:35:20', 130.15,'Order Cancelled:Customer'),
(7, 83,'2025-01-12 11:20:40', 100.00,'Product Returned:Customer'),
(8, 85,'2025-01-14 13:15:30',  89.99,'Order Cancelled:Customer'),
(9, 89,'2025-01-15 15:00:00',  78.50,'Payment Error:Retailer'),
(10,91,'2025-01-17 16:45:15', 250.75,'Product Returned:Customer');


num_affected_rows,num_inserted_rows
10,10


In [0]:
%sql
SELECT * FROM gizmobox.bronze.refunds

refund_id,payment_id,refund_timestamp,refund_amount,refund_reason
1,66,2025-01-10T11:30:00.000Z,85.75,Payment Error:Retailer
2,69,2025-01-01T12:40:15.000Z,120.50,Order Cancelled:Customer
3,72,2025-01-06T14:45:30.000Z,65.00,Product Returned:Customer
4,73,2025-01-07T16:10:45.000Z,210.99,Order Cancelled:Customer
5,75,2025-01-09T18:25:00.000Z,45.20,Payment Error:Retailer
6,80,2025-01-10T09:35:20.000Z,130.15,Order Cancelled:Customer
7,83,2025-01-12T11:20:40.000Z,100.00,Product Returned:Customer
8,85,2025-01-14T13:15:30.000Z,89.99,Order Cancelled:Customer
9,89,2025-01-15T15:00:00.000Z,78.50,Payment Error:Retailer
10,91,2025-01-17T16:45:15.000Z,250.75,Product Returned:Customer


#### 2- create external table 

In [0]:
%sql
create    table if not exists hive_metastore.bronze.refunds
using  jdbc
options (
url = "jdbc:sqlserver://gizmobox.database.windows.net:1433;database=gizmobox-db",
dbtable = 'refunds',
user = 'gizmoboxadm',
password = 'gizmobox1234567890'
);

In [0]:
%sql
select * from hive_metastore.bronze.refunds;

In [0]:
%sql
select * from gizmobox.bronze.refunds;

refund_id,payment_id,refund_timestamp,refund_amount,refund_reason
1,66,2025-01-10T11:30:00.000Z,85.75,Payment Error:Retailer
2,69,2025-01-01T12:40:15.000Z,120.50,Order Cancelled:Customer
3,72,2025-01-06T14:45:30.000Z,65.00,Product Returned:Customer
4,73,2025-01-07T16:10:45.000Z,210.99,Order Cancelled:Customer
5,75,2025-01-09T18:25:00.000Z,45.20,Payment Error:Retailer
6,80,2025-01-10T09:35:20.000Z,130.15,Order Cancelled:Customer
7,83,2025-01-12T11:20:40.000Z,100.00,Product Returned:Customer
8,85,2025-01-14T13:15:30.000Z,89.99,Order Cancelled:Customer
9,89,2025-01-15T15:00:00.000Z,78.50,Payment Error:Retailer
10,91,2025-01-17T16:45:15.000Z,250.75,Product Returned:Customer


In [0]:
%sql
desc extended gizmobox.bronze.refunds;

col_name,data_type,comment
refund_id,int,null
payment_id,int,null
refund_timestamp,timestamp,null
refund_amount,"decimal(10,2)",null
refund_reason,string,null
,,
# Delta Statistics Columns,,
Column Names,"refund_id, refund_timestamp, refund_amount, refund_reason, payment_id",
Column Selection Method,first-32,
,,
